In [1]:
%pip install prefect

  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached jsonschema-4.24.0-py3-none-any.whl.metadata (7.8 kB)
Using cached anyio-4.9.0-py3-none-any.whl (100 kB)
Using cached jsonschema-4.24.0-py3-none-any.whl (88 kB)
  Attempting uninstall: anyio
    Found existing installation: anyio 3.7.1
    Uninstalling anyio-3.7.1:
      Successfully uninstalled anyio-3.7.1
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 4.17.3
    Uninstalling jsonschema-4.17.3:
      Successfully uninstalled jsonschema-4.17.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [jsonschema]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentrons 8.4.0 requires anyio<4.0.0,>=3.6.1, but you have anyio 4.9.0 which is incompatible.
opentrons 8.4.0 requires jsonschema<4.18.0,>=3.0.1, but you have jsonschema 4.24.0 which is incompatible.
Note: yo

In [2]:
from prefect import flow


@flow
def hello():
    print("Hello from Prefect!")


# serve this flow
hello.serve(name="hello-serve")

Your flow 'hello' is being served and polling for scheduled runs!

To trigger a run for this flow, use the following command:

        $ prefect deployment run 'hello/hello-serve'

You can also run your flow via the Prefect UI: https://app.prefect.cloud/account/5b838504-64cf-4297-9b35-b881ac6169b3/workspace/d2718b4c-b49a-43ce-83c2-baf6fb3b9665/deployments/deployment/<coroutine object sync_compatible.<locals>.coroutine_wrapper.<locals>.ctx_call at 0x175fdf140>

RuntimeError: This event loop is already running